<a href="https://colab.research.google.com/github/imsomdev/auto_email_reply/blob/main/full_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers==2.2.2
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9daba80c1d540ca52428abac02dfc322a8f3dfa7d1acffb9de2e07f042e68d4a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.0/415.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.8 MB/s eta 

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 11.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 60.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.6 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-wzh092tw/overlay/local/bin
    changing mode of /tmp/pip-build-env-wzh092tw/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-wzh092tw/overlay/loca

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [ ]:
def create_prompt(email: str) -> str:
    prompt = f"here is an email: {email}. Summarize the mail in one line strictly only subject"
    prompt_template = f'''SYSTEM: you are an assistant, you help clients and employees. Behave like that
USER: {prompt}

ASSISTANT:
'''
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    return response

In [ ]:
def reply_prompt(context: str, email_content: str) -> str:
    prompt = f"this is the context{context} according to this create a crisp reply to this mail:{email_content}."
    prompt_template = f'''SYSTEM: you are an assistant, you help clients and employees. Make it full professional.
USER: {prompt}

ASSISTANT:
'''
    # Getting the mail ready for reply using es and recieved mail
    response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    return response

In [ ]:
email = "Subject: Project Epsilon Progress? Hello [Recipient's Name], Trust this finds you well. Could you provide a brief status on Project Epsilon? Best, [Your Name]"

In [ ]:
def extract_response(text: str) -> str:
    # Split the text on 'ASSISTANT:'
    parts = text.split("ASSISTANT:")

    # Check if the split resulted in at least two parts
    if len(parts) >= 2:
        # Return the second part after stripping leading and trailing whitespace
        return parts[1].strip()
    else:
        return ""

In [ ]:
prompt_template = create_prompt(email)
# print(prompt_template)
sender_context = extract_response(prompt_template["choices"][0]["text"])
print(sender_context)

Project Epsilon progress update requested.


In [ ]:
try:
    import json
    import os
    import uuid

    import pandas as pd
    import numpy as np

    import elasticsearch
    from elasticsearch import Elasticsearch
    from elasticsearch import helpers
    from sentence_transformers import SentenceTransformer, util
    from tqdm import tqdm
except Exception as e:
    print("Some Modules are Missing :{}".format(e))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# using ngrok expose my local host, using same el database in colab
es = Elasticsearch(
    hosts=[{
        'host': 'yoyo.ngrok.io',
        'port': 443,
        'scheme': 'https',
    }]
)
es.ping()

True

In [ ]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        _ = list(sentence_embeddings.flatten())
        encod_np_array = np.array(_)
        encod_list = encod_np_array.tolist()
        return encod_list

In [ ]:
#search here
helper_token = Tokenizer()
# INPUT = input("Enter the Input Query ")
token_vector = helper_token.get_token(sender_context)

query = {

  "knn": {
    "field": "vectors",
    "query_vector":token_vector,
    "k": 2,
    "num_candidates": 100
  },
  "_source": ["name", "status"]
}

res = es.search(index='project_status', body=query)

# Process the results
documents = [x['_source'] for x in res['hits']['hits']]
documents

<ipython-input-16-7ae88f177ca9>:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='project_status', body=query)


[{'name': 'Project Epsilon-2',
  'status': 'Project Epsilon-2: In progress with daily sprints. Using agile methodology.'},
 {'name': 'Project Epsilon',
  'status': 'Project Epsilon: In the design phase. The UX team is currently finalizing mockups based on client feedback.'}]

In [ ]:
# Convert each dictionary to a string and remove curly brackets
stringified_items = [str(item)[1:-1] for item in documents]

# Join all the items with a comma and a space
result_str = ', '.join(stringified_items)

print(result_str)

'name': 'Project Epsilon-2', 'status': 'Project Epsilon-2: In progress with daily sprints. Using agile methodology.', 'name': 'Project Epsilon', 'status': 'Project Epsilon: In the design phase. The UX team is currently finalizing mockups based on client feedback.'


In [ ]:
result = reply_prompt(result_str, email)
reply = extract_response(result["choices"][0]["text"])
print(reply)

Llama.generate: prefix-match hit


Hello [Recipient's Name], Hope all is going as planned with the project! Based on my knowledge of your previous communication and the data provided in the context name 'Project Epsilon-2', I can give you a quick update. Project Epsilon-2 is currently in progress using agile methodology, with daily sprints to ensure timely delivery and quality standards. The UX team has been finalizing mockups based on client feedback for Project Epsilon, which remains in the design phase. Please let me know if there are any questions or concerns you may have at this time. I'm here to assist! Best regards [Your Name].
